Instalando dependências

In [1]:
!pip install bibtexparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 230 kB/s 
  Created wheel for bibtexparser: filename=bibtexparser-1.4.0-py3-none-any.whl size=42447 sha256=33aa2299a9add8aa7085054131d316c78a9d12c68f71dadef9bb00a60239f586
  Stored in directory: /root/.cache/pip/wheels/0c/b1/30/eea6f0a0ecf1af0d19348e0f6928462817e9176e6b71059dfb
Successfully built bibtexparser


Importando as libs necessárias

In [2]:
import bibtexparser 
import pandas as pd
from google.colab import files
import io
import yaml

#Lendo arquivos de entrada

Carregando arquivos .bib para o notebook

In [3]:
uploaded = files.upload()                                                                                                        

Saving acm.bib to acm.bib
Saving acm_2.bib to acm_2.bib
Saving arquivo_config.yaml to arquivo_config.yaml
Saving IEEE_CITATIONS_01.bib to IEEE_CITATIONS_01.bib
Saving jcs_2020.csv to jcs_2020.csv
Saving ScienceDirect_citations_1665627129854.bib to ScienceDirect_citations_1665627129854.bib
Saving scimagojr 2020.csv to scimagojr 2020.csv


#Criando função que irá ser utilizada durante o código

In [4]:
def junta_arquivos(lista_dfs):
  lista_dfs_temp = None
  for arquivo in lista_dfs:
    if lista_dfs_temp is None:
      lista_dfs_temp = arquivo
    else:
      lista_dfs_temp = lista_dfs_temp.append(arquivo)
  return lista_dfs_temp



# Transformando bibtex em dataframe

In [5]:
files_input = ['acm.bib', 'acm_2.bib','IEEE_CITATIONS_01.bib', 'ScienceDirect_citations_1665627129854.bib']
files_transformed = []

In [6]:
for file_name  in files_input:
  with io.BytesIO(uploaded[file_name]) as bibtex_file:
    bib_database = bibtexparser.load(bibtex_file)
    df = pd.DataFrame(bib_database.entries)
    files_transformed.append(df)

# Analisando nome das colunas de cada arquivo, para entender se elas batem ou não

In [7]:
files_transformed[0].columns 

Index(['series', 'location', 'keywords', 'numpages', 'pages', 'booktitle',
       'abstract', 'doi', 'url', 'address', 'publisher', 'isbn', 'year',
       'title', 'author', 'ENTRYTYPE', 'ID', 'articleno', 'month', 'journal',
       'issn', 'number', 'volume', 'issue_date', 'note'],
      dtype='object')

In [8]:
files_transformed[1].columns

Index(['series', 'location', 'keywords', 'numpages', 'pages', 'booktitle',
       'abstract', 'doi', 'url', 'address', 'publisher', 'isbn', 'year',
       'title', 'author', 'ENTRYTYPE', 'ID', 'articleno', 'month', 'journal',
       'issn', 'number', 'volume', 'issue_date'],
      dtype='object')

In [9]:
files_transformed[2].columns

Index(['month', 'issn', 'doi', 'keywords', 'abstract', 'pages', 'number',
       'volume', 'year', 'title', 'booktitle', 'author', 'ENTRYTYPE', 'ID',
       'journal', 'url', 'isbn', 'publisher'],
      dtype='object')

In [10]:
files_transformed[3].columns

Index(['abstract', 'keywords', 'author', 'url', 'doi', 'issn', 'note', 'year',
       'pages', 'volume', 'journal', 'title', 'ENTRYTYPE', 'ID', 'number',
       'booktitle', 'publisher', 'series', 'editor', 'isbn', 'address',
       'edition'],
      dtype='object')

#Preparando os df para conseguir juntá-los no futuro

Nosso objetivo é deixar as colunas com o seguinte cabeçalho: author, title, keywords, abstract, year, type_publication, doi

Tratando e selecionando apenas as colunas necessárias dos dois arquivos da fonte ACM - https://dl.acm.org/

In [11]:
acm1_file = files_transformed[0][["author", "title", "keywords", "abstract","year", "ENTRYTYPE","doi"]]
acm2_file = files_transformed[1][["author", "title", "keywords", "abstract","year", "ENTRYTYPE","doi"]]

acm_file = acm1_file.append(acm2_file)

Criando a coluna source para identificar de que fonte veio o arquivo

In [12]:
acm_file["source"] = "ACM"

Tratando e selecionando apenas as colunas necessárias do arquivo da fonte IEEE - https://ieeexplore.ieee.org/Xplore/home.jsp

In [13]:
ieee_file = files_transformed[2][["author", "title", "keywords", "abstract","year", "ENTRYTYPE","doi"]]

Criando a coluna source para identificar de que fonte veio o arquivo

In [14]:
ieee_file["source"] = "IEEE"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Tratando e selecionando apenas as colunas necessárias do arquivo da fonte Science Direct - https://www.sciencedirect.com/


In [15]:
science_direct_file = files_transformed[3][["author", "title", "keywords", "abstract","year", "ENTRYTYPE","doi"]]

Criando a coluna source para identificar de que fonte veio o arquivo

In [16]:
science_direct_file["source"] = "ScienceDirect"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Visualizando os arquivos após selecionado as colunas necessárias

In [17]:
display(acm_file)

,author,title,keywords,abstract,year,ENTRYTYPE,doi,source
0,"El Alaoui, Imane and Gahi, Youssef and Messous...",Big Data Quality Metrics for Sentiment Analysi...,"Sentiment analysis, Big data quality metrics, ...","In a world increasingly connected, and in whic...",2019,inproceedings,10.1145/3341620.3341629,ACM
1,"Reda, Oumaima and Sassi, Imad and Zellou, Ahme...",Towards a Data Quality Assessment in Big Data,"Data Quality evaluation, Big Data, Quality Mod...","In recent years, as more and more data sources...",2020,inproceedings,10.1145/3419604.3419803,ACM
2,"Baldassarre, Maria Teresa and Caballero, Ismae...",From Big Data to Smart Data: A Data Quality Pe...,"Big Data, Data Quality, Smart Data",Big Data (BD) solutions are designed to better...,2018,inproceedings,10.1145/3281022.3281026,ACM
3,"Emmanuel, Isitor and Stanier, Clare",Defining Big Data,"Big Data characteristics, Data Quality Dimensi...","As Big Data becomes better understood, there i...",2016,inproceedings,10.1145/3010089.3010090,ACM
4,"Cuzzocrea, Alfredo and Sacc\`{a}, Domenico and...",Big Data: A Research Agenda,"big data, OLAP over big data, privacy of big d...","Recently, a great deal of interest for Big Dat...",2013,inproceedings,10.1145/2513591.2527071,ACM
...,...,...,...,...,...,...,...,...
45,"Zhichao, Xu and Jiandong, Zhao and Huan, Huang",Based on Hadoop's Tech Big Data Combination an...,"tech big data, Hadoop, combination, mining",With the advent of the Internet + era in the f...,2018,inproceedings,10.1145/3194206.3194229,ACM
46,"Wu, Min and Hao, Xinxin and Wan, Xuehong and M...",Opportunities and Challenges of Joint Training...,"big data, cooperative education of university-...",University-industry cooperative education is a...,2022,inproceedings,10.1145/3524383.3524431,ACM
47,"Bizer, Christian and Boncz, Peter and Brodie, ...",The Meaningful Use of Big Data: Four Perspecti...,NaN,Twenty-five Semantic Web and Database research...,2012,article,10.1145/2094114.2094129,ACM
48,"Yu, Bangbo and Zhao, Haijun",Research on the Construction of Big Data Tradi...,"regulatory construction, big data trading plat...","As a new type of asset, the value of big data ...",2019,inproceedings,10.1145/3321454.3321474,ACM


In [18]:
display(ieee_file)

,author,title,keywords,abstract,year,ENTRYTYPE,doi,source
0,"Liu, He and Huang, Fupeng and Li, Han and Liu,...",A Big Data Framework for Electric Power Data Q...,Big Data;Data integrity;Power grids;History;Re...,Since a low-quality data may influence the eff...,2017,inproceedings,10.1109/WISA.2017.29,IEEE
1,"Taleb, Ikbal and Serhani, Mohamed Adel",Big Data Pre-Processing: Closing the Data Qual...,Big Data;Optimization;Data models;Quality asse...,"In the Big Data Era, data is the core for any ...",2017,inproceedings,10.1109/BigDataCongress.2017.73,IEEE
2,"Loetpipatwanich, Sakda and Vichitthamaros, Pre...",Sakdas: A Python Package for Data Profiling an...,Data integrity;Pipelines;Data visualization;Bi...,Data Profiling and data quality management bec...,2020,inproceedings,10.1109/IBDAP50342.2020.9245455,IEEE
3,"Taleb, Ikbal and Serhani, Mohamed Adel and Dss...",Big Data Quality Assessment Model for Unstruct...,Big Data;Data integrity;Data mining;Feature ex...,Big Data has gained an enormous momentum the p...,2018,inproceedings,10.1109/INNOVATIONS.2018.8605945,IEEE
4,"HongJu, Xiao and Fei, Wang and FenMei, Wang an...",Some key problems of data management in army d...,Data engineering;Data analysis;Data integratio...,This paper analyzed the challenges of data man...,2017,inproceedings,10.1109/ICBDA.2017.8078796,IEEE
...,...,...,...,...,...,...,...,...
95,"Wang, Jianwu and Tang, Yan and Nguyen, Mai and...",A Scalable Data Science Workflow Approach for ...,Big data;Engines;Bayes methods;Partitioning al...,"In the Big Data era, machine learning has more...",2014,inproceedings,10.1109/BDC.2014.10,IEEE
96,"Ahnn, Jong Hoon",A Practical Approach to Scalable Big Data Comp...,Adaptation models;Speech;Acoustics;Computation...,We observe that the recent advances in big dat...,2014,inproceedings,10.1109/BDC.2014.11,IEEE
97,"Ludwig, Heiko",Managing Big Data Effectively - A Cloud Provid...,Big data;Cloud computing;Organizations;Face;Co...,Summary form only given. Instrumentation of pr...,2014,inproceedings,10.1109/EDOC.2014.21,IEEE
98,"Ding, Junhua and Kang, Xiaojun and Hu, Xin-Hua...",Building a Deep Learning Classifier for Enhanc...,Diffraction;Big Data;Machine learning;Morpholo...,Providing an easily accessible data service wi...,2017,inproceedings,10.1109/SCC.2017.25,IEEE


In [19]:
display(science_direct_file)

,author,title,keywords,abstract,year,ENTRYTYPE,doi,source
0,Jorge Merino and Ismael Caballero and Bibiano ...,A Data Quality in Use model for Big Data,"Data Quality, Big Data, Measurement, Quality-i...","Beyond the hype of Big Data, something within ...",2016,article,https://doi.org/10.1016/j.future.2015.11.024,ScienceDirect
1,Xintao Liu and Min Chen and Christophe Claramu...,Geographic information science in the era of g...,NaN,NaN,2022,article,https://doi.org/10.1016/j.xinn.2022.100279,ScienceDirect
2,Yuri A.W. Shardt and Xu Yang and Kevin Brooks ...,Data Quality Assessment for System Identificat...,"data quality assessment, system identification...",As the amount of data stored from industrial p...,2020,article,https://doi.org/10.1016/j.ifacol.2020.12.103,ScienceDirect
3,Yi-Chia Lee and Ying-Ting Chao and Pei-Ju Lin ...,Quality assurance of integrative big data for ...,"Big data, Electronic health record, Evidence b...",Background\nThe need is growing to create medi...,2022,article,https://doi.org/10.1016/j.jfma.2021.12.024,ScienceDirect
4,Conor John Cremin and Sabyasachi Dash and Xiao...,Big data: Historic advances and emerging trend...,"Big data, Big data in biomedicine, Data analyt...",Big data is transforming biomedical research b...,2022,article,https://doi.org/10.1016/j.crbiot.2022.02.004,ScienceDirect
...,...,...,...,...,...,...,...,...
95,Angelo Corallo and Anna Maria Crespino and Mar...,Model-based Big Data Analytics-as-a-Service fr...,"Big Data Analytics, BDA, MBDAaaS framework, Sm...","Today, in a smart manufacturing environment ba...",2022,article,https://doi.org/10.1016/j.rcim.2022.102331,ScienceDirect
96,Dirk Schmücker and Julian Reif,Measuring tourism with big data? Empirical ins...,"Big Data, Mobile Network Data, Passive GPS Dat...",In this paper we aim to classify digital data ...,2022,article,https://doi.org/10.1016/j.annale.2022.100061,ScienceDirect
97,Nikolay Nikolov and Yared Dejene Dessalk and A...,Conceptualization and scalable execution of bi...,"Big data workflows, Internet of Things, Domain...","Big Data processing, especially with the incre...",2021,article,https://doi.org/10.1016/j.iot.2021.100440,ScienceDirect
98,Yuncheng Shen and Bing Guo and Yan Shen and Xu...,Personal big data pricing method based on diff...,"Personal big data, Data privacy, Privacy prote...",Personal big data can greatly promote social m...,2022,article,https://doi.org/10.1016/j.cose.2021.102529,ScienceDirect


Juntando os arquivos em apenas um df

In [20]:
arquivos_entrada = junta_arquivos([acm_file, ieee_file, science_direct_file])

In [21]:
display(arquivos_entrada)

,author,title,keywords,abstract,year,ENTRYTYPE,doi,source
0,"El Alaoui, Imane and Gahi, Youssef and Messous...",Big Data Quality Metrics for Sentiment Analysi...,"Sentiment analysis, Big data quality metrics, ...","In a world increasingly connected, and in whic...",2019,inproceedings,10.1145/3341620.3341629,ACM
1,"Reda, Oumaima and Sassi, Imad and Zellou, Ahme...",Towards a Data Quality Assessment in Big Data,"Data Quality evaluation, Big Data, Quality Mod...","In recent years, as more and more data sources...",2020,inproceedings,10.1145/3419604.3419803,ACM
2,"Baldassarre, Maria Teresa and Caballero, Ismae...",From Big Data to Smart Data: A Data Quality Pe...,"Big Data, Data Quality, Smart Data",Big Data (BD) solutions are designed to better...,2018,inproceedings,10.1145/3281022.3281026,ACM
3,"Emmanuel, Isitor and Stanier, Clare",Defining Big Data,"Big Data characteristics, Data Quality Dimensi...","As Big Data becomes better understood, there i...",2016,inproceedings,10.1145/3010089.3010090,ACM
4,"Cuzzocrea, Alfredo and Sacc\`{a}, Domenico and...",Big Data: A Research Agenda,"big data, OLAP over big data, privacy of big d...","Recently, a great deal of interest for Big Dat...",2013,inproceedings,10.1145/2513591.2527071,ACM
...,...,...,...,...,...,...,...,...
95,Angelo Corallo and Anna Maria Crespino and Mar...,Model-based Big Data Analytics-as-a-Service fr...,"Big Data Analytics, BDA, MBDAaaS framework, Sm...","Today, in a smart manufacturing environment ba...",2022,article,https://doi.org/10.1016/j.rcim.2022.102331,ScienceDirect
96,Dirk Schmücker and Julian Reif,Measuring tourism with big data? Empirical ins...,"Big Data, Mobile Network Data, Passive GPS Dat...",In this paper we aim to classify digital data ...,2022,article,https://doi.org/10.1016/j.annale.2022.100061,ScienceDirect
97,Nikolay Nikolov and Yared Dejene Dessalk and A...,Conceptualization and scalable execution of bi...,"Big data workflows, Internet of Things, Domain...","Big Data processing, especially with the incre...",2021,article,https://doi.org/10.1016/j.iot.2021.100440,ScienceDirect
98,Yuncheng Shen and Bing Guo and Yan Shen and Xu...,Personal big data pricing method based on diff...,"Personal big data, Data privacy, Privacy prote...",Personal big data can greatly promote social m...,2022,article,https://doi.org/10.1016/j.cose.2021.102529,ScienceDirect


#Renomeando nome de coluna



In [22]:
arquivos_entrada = arquivos_entrada.rename(columns={'ENTRYTYPE': 'type_publication'})

In [23]:
display(arquivos_entrada)

,author,title,keywords,abstract,year,type_publication,doi,source
0,"El Alaoui, Imane and Gahi, Youssef and Messous...",Big Data Quality Metrics for Sentiment Analysi...,"Sentiment analysis, Big data quality metrics, ...","In a world increasingly connected, and in whic...",2019,inproceedings,10.1145/3341620.3341629,ACM
1,"Reda, Oumaima and Sassi, Imad and Zellou, Ahme...",Towards a Data Quality Assessment in Big Data,"Data Quality evaluation, Big Data, Quality Mod...","In recent years, as more and more data sources...",2020,inproceedings,10.1145/3419604.3419803,ACM
2,"Baldassarre, Maria Teresa and Caballero, Ismae...",From Big Data to Smart Data: A Data Quality Pe...,"Big Data, Data Quality, Smart Data",Big Data (BD) solutions are designed to better...,2018,inproceedings,10.1145/3281022.3281026,ACM
3,"Emmanuel, Isitor and Stanier, Clare",Defining Big Data,"Big Data characteristics, Data Quality Dimensi...","As Big Data becomes better understood, there i...",2016,inproceedings,10.1145/3010089.3010090,ACM
4,"Cuzzocrea, Alfredo and Sacc\`{a}, Domenico and...",Big Data: A Research Agenda,"big data, OLAP over big data, privacy of big d...","Recently, a great deal of interest for Big Dat...",2013,inproceedings,10.1145/2513591.2527071,ACM
...,...,...,...,...,...,...,...,...
95,Angelo Corallo and Anna Maria Crespino and Mar...,Model-based Big Data Analytics-as-a-Service fr...,"Big Data Analytics, BDA, MBDAaaS framework, Sm...","Today, in a smart manufacturing environment ba...",2022,article,https://doi.org/10.1016/j.rcim.2022.102331,ScienceDirect
96,Dirk Schmücker and Julian Reif,Measuring tourism with big data? Empirical ins...,"Big Data, Mobile Network Data, Passive GPS Dat...",In this paper we aim to classify digital data ...,2022,article,https://doi.org/10.1016/j.annale.2022.100061,ScienceDirect
97,Nikolay Nikolov and Yared Dejene Dessalk and A...,Conceptualization and scalable execution of bi...,"Big data workflows, Internet of Things, Domain...","Big Data processing, especially with the incre...",2021,article,https://doi.org/10.1016/j.iot.2021.100440,ScienceDirect
98,Yuncheng Shen and Bing Guo and Yan Shen and Xu...,Personal big data pricing method based on diff...,"Personal big data, Data privacy, Privacy prote...",Personal big data can greatly promote social m...,2022,article,https://doi.org/10.1016/j.cose.2021.102529,ScienceDirect


#Verificando se existe caso de nulos

In [24]:
arquivos_entrada.isnull().all()

author              False
title               False
keywords            False
abstract            False
year                False
type_publication    False
doi                 False
source              False
dtype: bool

# Tratando dados para deixar mais padrao para o tipo de coluna (ainda vou construir essa parte)

In [25]:
arquivos_entrada

,author,title,keywords,abstract,year,type_publication,doi,source
0,"El Alaoui, Imane and Gahi, Youssef and Messous...",Big Data Quality Metrics for Sentiment Analysi...,"Sentiment analysis, Big data quality metrics, ...","In a world increasingly connected, and in whic...",2019,inproceedings,10.1145/3341620.3341629,ACM
1,"Reda, Oumaima and Sassi, Imad and Zellou, Ahme...",Towards a Data Quality Assessment in Big Data,"Data Quality evaluation, Big Data, Quality Mod...","In recent years, as more and more data sources...",2020,inproceedings,10.1145/3419604.3419803,ACM
2,"Baldassarre, Maria Teresa and Caballero, Ismae...",From Big Data to Smart Data: A Data Quality Pe...,"Big Data, Data Quality, Smart Data",Big Data (BD) solutions are designed to better...,2018,inproceedings,10.1145/3281022.3281026,ACM
3,"Emmanuel, Isitor and Stanier, Clare",Defining Big Data,"Big Data characteristics, Data Quality Dimensi...","As Big Data becomes better understood, there i...",2016,inproceedings,10.1145/3010089.3010090,ACM
4,"Cuzzocrea, Alfredo and Sacc\`{a}, Domenico and...",Big Data: A Research Agenda,"big data, OLAP over big data, privacy of big d...","Recently, a great deal of interest for Big Dat...",2013,inproceedings,10.1145/2513591.2527071,ACM
...,...,...,...,...,...,...,...,...
95,Angelo Corallo and Anna Maria Crespino and Mar...,Model-based Big Data Analytics-as-a-Service fr...,"Big Data Analytics, BDA, MBDAaaS framework, Sm...","Today, in a smart manufacturing environment ba...",2022,article,https://doi.org/10.1016/j.rcim.2022.102331,ScienceDirect
96,Dirk Schmücker and Julian Reif,Measuring tourism with big data? Empirical ins...,"Big Data, Mobile Network Data, Passive GPS Dat...",In this paper we aim to classify digital data ...,2022,article,https://doi.org/10.1016/j.annale.2022.100061,ScienceDirect
97,Nikolay Nikolov and Yared Dejene Dessalk and A...,Conceptualization and scalable execution of bi...,"Big data workflows, Internet of Things, Domain...","Big Data processing, especially with the incre...",2021,article,https://doi.org/10.1016/j.iot.2021.100440,ScienceDirect
98,Yuncheng Shen and Bing Guo and Yan Shen and Xu...,Personal big data pricing method based on diff...,"Personal big data, Data privacy, Privacy prote...",Personal big data can greatly promote social m...,2022,article,https://doi.org/10.1016/j.cose.2021.102529,ScienceDirect


In [26]:
arquivos_entrada['doi'] = arquivos_entrada['doi'].str.replace(r'https://doi.org/', '')



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [27]:
arquivos_entrada

,author,title,keywords,abstract,year,type_publication,doi,source
0,"El Alaoui, Imane and Gahi, Youssef and Messous...",Big Data Quality Metrics for Sentiment Analysi...,"Sentiment analysis, Big data quality metrics, ...","In a world increasingly connected, and in whic...",2019,inproceedings,10.1145/3341620.3341629,ACM
1,"Reda, Oumaima and Sassi, Imad and Zellou, Ahme...",Towards a Data Quality Assessment in Big Data,"Data Quality evaluation, Big Data, Quality Mod...","In recent years, as more and more data sources...",2020,inproceedings,10.1145/3419604.3419803,ACM
2,"Baldassarre, Maria Teresa and Caballero, Ismae...",From Big Data to Smart Data: A Data Quality Pe...,"Big Data, Data Quality, Smart Data",Big Data (BD) solutions are designed to better...,2018,inproceedings,10.1145/3281022.3281026,ACM
3,"Emmanuel, Isitor and Stanier, Clare",Defining Big Data,"Big Data characteristics, Data Quality Dimensi...","As Big Data becomes better understood, there i...",2016,inproceedings,10.1145/3010089.3010090,ACM
4,"Cuzzocrea, Alfredo and Sacc\`{a}, Domenico and...",Big Data: A Research Agenda,"big data, OLAP over big data, privacy of big d...","Recently, a great deal of interest for Big Dat...",2013,inproceedings,10.1145/2513591.2527071,ACM
...,...,...,...,...,...,...,...,...
95,Angelo Corallo and Anna Maria Crespino and Mar...,Model-based Big Data Analytics-as-a-Service fr...,"Big Data Analytics, BDA, MBDAaaS framework, Sm...","Today, in a smart manufacturing environment ba...",2022,article,10.1016/j.rcim.2022.102331,ScienceDirect
96,Dirk Schmücker and Julian Reif,Measuring tourism with big data? Empirical ins...,"Big Data, Mobile Network Data, Passive GPS Dat...",In this paper we aim to classify digital data ...,2022,article,10.1016/j.annale.2022.100061,ScienceDirect
97,Nikolay Nikolov and Yared Dejene Dessalk and A...,Conceptualization and scalable execution of bi...,"Big data workflows, Internet of Things, Domain...","Big Data processing, especially with the incre...",2021,article,10.1016/j.iot.2021.100440,ScienceDirect
98,Yuncheng Shen and Bing Guo and Yan Shen and Xu...,Personal big data pricing method based on diff...,"Personal big data, Data privacy, Privacy prote...",Personal big data can greatly promote social m...,2022,article,10.1016/j.cose.2021.102529,ScienceDirect


# Lendo arquivo .yaml de configuração

In [28]:
with open("arquivo_config.yaml", "r") as stream:
  arquivo_config = yaml.safe_load(stream)
    

In [29]:
arquivo_config

{'output': ['json', 'csv', 'yaml', 'xml']}

In [30]:
for tipo in arquivo_config['output']:
  print(tipo)

json
csv
yaml
xml


# Gerando arquivo conforme configuração do .yaml

In [31]:
#csv
if 'csv' in arquivo_config['output']:
  arquivos_entrada.to_csv('resultado.csv')


In [32]:
#json
if 'json' in arquivo_config['output']:
  arquivos_entrada.to_json('resultado.json', orient="records")


In [33]:
#yaml
if 'yaml' in arquivo_config['output']:
  yaml_arquivos_entrada = yaml.dump(arquivos_entrada.to_dict("records"))
  escrita = open("resultado.yaml", "w")
  escrita.write(yaml_arquivos_entrada)
  escrita.close()



In [34]:
#xml

if 'xml' in arquivo_config['output']:
  arquivos_entrada.to_xml('resultado.xml')
  




#Parte 2

---






###Leitura arquivos csv

In [35]:
df = pd.read_csv("jcs_2020.csv", sep=';') 

In [36]:
df2 = pd.read_csv("scimagojr 2020.csv", sep= ';') 

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [37]:
df.count()

Rank                     13010
Full Journal Title       13010
Total Cites              13010
Unnamed: 3                   0
Journal Impact Factor    13010
Eigenfactor Score        13010
Unnamed: 6                   0
Unnamed: 7                   0
dtype: int64

In [38]:
df2.count()

Rank                      32952
Sourceid                  32952
Title                     32952
Type                      32952
Issn                      32952
SJR                       32604
SJR Best Quartile         32952
H index                   32952
Total Docs. (2020)        32952
Total Docs. (3years)      32952
Total Refs.               32952
Total Cites (3years)      32952
Citable Docs. (3years)    32952
Cites / Doc. (2years)     32952
Ref. / Doc.               32952
Country                   32952
Region                    32952
Publisher                 26048
Coverage                  32952
Categories                32952
dtype: int64

In [39]:
df.head()

,Rank,Full Journal Title,Total Cites,Unnamed: 3,Journal Impact Factor,Eigenfactor Score,Unnamed: 6,Unnamed: 7
0,1,CA-A CANCER JOURNAL FOR CLINICIANS,"55,868",NaN,508.702,0.10514,NaN,NaN
1,2,NATURE REVIEWS MOLECULAR CELL BIOLOGY,"58,477",NaN,94.444,0.07548,NaN,NaN
2,3,NEW ENGLAND JOURNAL OF MEDICINE,"464,351",NaN,91.245,0.63118,NaN,NaN
3,4,NATURE REVIEWS DRUG DISCOVERY,"41,989",NaN,84.694,0.04822,NaN,NaN
4,5,LANCET,"369,601",NaN,79.321,0.44524,NaN,NaN


In [40]:
df2.head()

,Rank,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2020),Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories
0,1,28773,Ca-A Cancer Journal for Clinicians,journal,"15424863, 00079235","62,937",Q1,168,47,119,3452,15499,80,"126,34","73,45",United States,Northern America,Wiley-Blackwell,1950-2020,Hematology (Q1); Oncology (Q1)
1,2,19434,MMWR Recommendations and Reports,journal,"10575987, 15458601","40,949",Q1,143,10,9,1292,492,9,"50,00","129,20",United States,Northern America,Centers for Disease Control and Prevention (CDC),1990-2020,Epidemiology (Q1); Health Information Manageme...
2,3,20315,Nature Reviews Molecular Cell Biology,journal,"14710072, 14710080","37,461",Q1,431,115,338,8439,10844,167,"32,83","73,38",United Kingdom,Western Europe,Nature Publishing Group,2000-2020,Cell Biology (Q1); Molecular Biology (Q1)
3,4,29431,Quarterly Journal of Economics,journal,"00335533, 15314650","34,573",Q1,259,40,110,2733,1945,109,"16,00","68,33",United Kingdom,Western Europe,Oxford University Press,1886-2020,Economics and Econometrics (Q1)
4,5,21100812243,Nature Reviews Materials,journal,20588437,"32,011",Q1,108,92,264,10632,11188,138,"32,15","115,57",United Kingdom,Western Europe,Nature Publishing Group,2016-2020,"Biomaterials (Q1); Electronic, Optical and Mag..."


#Removendo colunas que não possui dados

In [41]:
df.isnull().all()

Rank                     False
Full Journal Title       False
Total Cites              False
Unnamed: 3                True
Journal Impact Factor    False
Eigenfactor Score        False
Unnamed: 6                True
Unnamed: 7                True
dtype: bool

In [42]:
df2.isnull().all()

Rank                      False
Sourceid                  False
Title                     False
Type                      False
Issn                      False
SJR                       False
SJR Best Quartile         False
H index                   False
Total Docs. (2020)        False
Total Docs. (3years)      False
Total Refs.               False
Total Cites (3years)      False
Citable Docs. (3years)    False
Cites / Doc. (2years)     False
Ref. / Doc.               False
Country                   False
Region                    False
Publisher                 False
Coverage                  False
Categories                False
dtype: bool

In [43]:
df = df[['Rank', 'Full Journal Title','Total Cites', 'Journal Impact Factor', 'Eigenfactor Score']]

In [44]:
df.head()

,Rank,Full Journal Title,Total Cites,Journal Impact Factor,Eigenfactor Score
0,1,CA-A CANCER JOURNAL FOR CLINICIANS,"55,868",508.702,0.10514
1,2,NATURE REVIEWS MOLECULAR CELL BIOLOGY,"58,477",94.444,0.07548
2,3,NEW ENGLAND JOURNAL OF MEDICINE,"464,351",91.245,0.63118
3,4,NATURE REVIEWS DRUG DISCOVERY,"41,989",84.694,0.04822
4,5,LANCET,"369,601",79.321,0.44524


#Relacionando os dois novos .csv

In [45]:
df['Full Journal Title'] = df['Full Journal Title'].str.capitalize()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
df2['Title'] = df2['Title'].str.capitalize()

In [47]:
novo_dataframe = pd.merge(df2, df, left_on=['Title'], right_on=['Full Journal Title'], how='left')

In [48]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [49]:
display(novo_dataframe)

,Rank_x,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2020),Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories,Rank_y,Full Journal Title,Total Cites,Journal Impact Factor,Eigenfactor Score
0,1,28773,Ca-a cancer journal for clinicians,journal,"15424863, 00079235","62,937",Q1,168,47,119,3452,15499,80,"126,34","73,45",United States,Northern America,Wiley-Blackwell,1950-2020,Hematology (Q1); Oncology (Q1),1.0,Ca-a cancer journal for clinicians,"55,868",508.702,0.10514
1,2,19434,Mmwr recommendations and reports,journal,"10575987, 15458601","40,949",Q1,143,10,9,1292,492,9,"50,00","129,20",United States,Northern America,Centers for Disease Control and Prevention (CDC),1990-2020,Epidemiology (Q1); Health Information Manageme...,14.0,Mmwr recommendations and reports,"3,288",55.857,0.01123
2,3,20315,Nature reviews molecular cell biology,journal,"14710072, 14710080","37,461",Q1,431,115,338,8439,10844,167,"32,83","73,38",United Kingdom,Western Europe,Nature Publishing Group,2000-2020,Cell Biology (Q1); Molecular Biology (Q1),2.0,Nature reviews molecular cell biology,"58,477",94.444,0.07548
3,4,29431,Quarterly journal of economics,journal,"00335533, 15314650","34,573",Q1,259,40,110,2733,1945,109,"16,00","68,33",United Kingdom,Western Europe,Oxford University Press,1886-2020,Economics and Econometrics (Q1),215.0,Quarterly journal of economics,"38,427",15.563,0.06169
4,5,21100812243,Nature reviews materials,journal,20588437,"32,011",Q1,108,92,264,10632,11188,138,"32,15","115,57",United Kingdom,Western Europe,Nature Publishing Group,2016-2020,"Biomaterials (Q1); Electronic, Optical and Mag...",7.0,Nature reviews materials,"19,887",66.308,0.05677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33506,32948,25412,Zhonghua kou qiang yi xue za zhi = zhonghua ko...,journal,10020098,NaN,-,14,150,0,0,0,0,"0,00","0,00",China,Asiatic Region,Zhonghua Yixuehui Zazhishe,"1987-2016, 2020",Medicine (miscellaneous),NaN,NaN,NaN,NaN,NaN
33507,32949,21485,Zhonghua liu xing bing xue za zhi = zhonghua l...,journal,02546450,NaN,-,31,292,0,0,0,0,"0,00","0,00",China,Asiatic Region,Zhonghua Yixuehui Zazhishe,"1982-2016, 2020",Medicine (miscellaneous),NaN,NaN,NaN,NaN,NaN
33508,32950,26726,Zhonghua nei ke za zhi [chinese journal of int...,journal,05781426,NaN,-,18,5,0,0,0,0,"0,00","0,00",China,Asiatic Region,Zhonghua Yixuehui Zazhishe,"1957-1959, 1979-1997, 1999-2016, 2020",Medicine (miscellaneous),NaN,NaN,NaN,NaN,NaN
33509,32951,19324,Zhonghua wai ke za zhi [chinese journal of sur...,journal,05295815,NaN,-,16,5,0,0,0,0,"0,00","0,00",China,Asiatic Region,Zhonghua Yixuehui Zazhishe,"1957, 1959-1964, 1979-2016, 2020",Medicine (miscellaneous),NaN,NaN,NaN,NaN,NaN


In [50]:
display(arquivos_entrada)

,author,title,keywords,abstract,year,type_publication,doi,source
0,"El Alaoui, Imane and Gahi, Youssef and Messous...",Big Data Quality Metrics for Sentiment Analysi...,"Sentiment analysis, Big data quality metrics, ...","In a world increasingly connected, and in whic...",2019,inproceedings,10.1145/3341620.3341629,ACM
1,"Reda, Oumaima and Sassi, Imad and Zellou, Ahme...",Towards a Data Quality Assessment in Big Data,"Data Quality evaluation, Big Data, Quality Mod...","In recent years, as more and more data sources...",2020,inproceedings,10.1145/3419604.3419803,ACM
2,"Baldassarre, Maria Teresa and Caballero, Ismae...",From Big Data to Smart Data: A Data Quality Pe...,"Big Data, Data Quality, Smart Data",Big Data (BD) solutions are designed to better...,2018,inproceedings,10.1145/3281022.3281026,ACM
3,"Emmanuel, Isitor and Stanier, Clare",Defining Big Data,"Big Data characteristics, Data Quality Dimensi...","As Big Data becomes better understood, there i...",2016,inproceedings,10.1145/3010089.3010090,ACM
4,"Cuzzocrea, Alfredo and Sacc\`{a}, Domenico and...",Big Data: A Research Agenda,"big data, OLAP over big data, privacy of big d...","Recently, a great deal of interest for Big Dat...",2013,inproceedings,10.1145/2513591.2527071,ACM
5,"Neamtu, Rodica and Ahsan, Ramoza and Stokes, J...",Taming Big Data: Integrating Diverse Public Da...,"Big data, data integration, diverse data sources",In an era where Big Data can greatly impact a ...,2014,inproceedings,10.1145/2658840.2658845,ACM
6,"Pti\v{c}ek, Marina and Vrdoljak, Boris",Big Data and New Data Warehousing Approaches,"big data, NoSQL, databases, NewSQL, data wareh...",Big data are a data trend present around us ma...,2017,inproceedings,10.1145/3141128.3141139,ACM
7,"Davoudian, Ali and Liu, Mengchi",Big Data Systems: A Software Engineering Persp...,"Big Data, software engineering, quality assura...",Big Data Systems (BDSs) are an emerging class ...,2020,article,10.1145/3408314,ACM
8,"Abell\'{o}, Alberto",Big Data Design,"nosql, database design, big data",It is widely accepted today that Relational da...,2015,inproceedings,10.1145/2811222.2811235,ACM
9,"Safhi, Hicham Moad and Frikh, Bouchra and Ouhb...",Data Source Selection in Big Data Context,"Big Data Source Selection, Data quality, Sourc...",Big Data presents promising technological and ...,2019,inproceedings,10.1145/3366030.3366121,ACM


In [51]:
novo_dataframe['Issn'] = novo_dataframe['Issn'].astype(str)
novo_dataframe['Issn'] = novo_dataframe['Issn'].str.replace(',', '.')
novo_dataframe['Issn'] = novo_dataframe['Issn'].str.replace(' ', '')

In [52]:
arquivos_entrada['doi_issn'] = arquivos_entrada['doi'].str[-14:]

In [53]:
arquivo_final = pd.merge(arquivos_entrada, novo_dataframe, left_on=['doi_issn'], right_on=['Issn'], how='left')

In [54]:
display(arquivo_final)

,author,title,keywords,abstract,year,type_publication,doi,source,doi_issn,Rank_x,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2020),Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories,Rank_y,Full Journal Title,Total Cites,Journal Impact Factor,Eigenfactor Score
0,"El Alaoui, Imane and Gahi, Youssef and Messous...",Big Data Quality Metrics for Sentiment Analysi...,"Sentiment analysis, Big data quality metrics, ...","In a world increasingly connected, and in whic...",2019,inproceedings,10.1145/3341620.3341629,ACM,341620.3341629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Reda, Oumaima and Sassi, Imad and Zellou, Ahme...",Towards a Data Quality Assessment in Big Data,"Data Quality evaluation, Big Data, Quality Mod...","In recent years, as more and more data sources...",2020,inproceedings,10.1145/3419604.3419803,ACM,419604.3419803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Baldassarre, Maria Teresa and Caballero, Ismae...",From Big Data to Smart Data: A Data Quality Pe...,"Big Data, Data Quality, Smart Data",Big Data (BD) solutions are designed to better...,2018,inproceedings,10.1145/3281022.3281026,ACM,281022.3281026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Emmanuel, Isitor and Stanier, Clare",Defining Big Data,"Big Data characteristics, Data Quality Dimensi...","As Big Data becomes better understood, there i...",2016,inproceedings,10.1145/3010089.3010090,ACM,010089.3010090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Cuzzocrea, Alfredo and Sacc\`{a}, Domenico and...",Big Data: A Research Agenda,"big data, OLAP over big data, privacy of big d...","Recently, a great deal of interest for Big Dat...",2013,inproceedings,10.1145/2513591.2527071,ACM,513591.2527071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Neamtu, Rodica and Ahsan, Ramoza and Stokes, J...",Taming Big Data: Integrating Diverse Public Da...,"Big data, data integration, diverse data sources",In an era where Big Data can greatly impact a ...,2014,inproceedings,10.1145/2658840.2658845,ACM,658840.2658845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"Pti\v{c}ek, Marina and Vrdoljak, Boris",Big Data and New Data Warehousing Approaches,"big data, NoSQL, databases, NewSQL, data wareh...",Big data are a data trend present around us ma...,2017,inproceedings,10.1145/3141128.3141139,ACM,141128.3141139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"Davoudian, Ali and Liu, Mengchi",Big Data Systems: A Software Engineering Persp...,"Big Data, software engineering, quality assura...",Big Data Systems (BDSs) are an emerging class ...,2020,article,10.1145/3408314,ACM,0.1145/3408314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"Abell\'{o}, Alberto",Big Data Design,"nosql, database design, big data",It is widely accepted today that Relational da...,2015,inproceedings,10.1145/2811222.2811235,ACM,811222.2811235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"Safhi, Hicham Moad and Frikh, Bouchra and Ouhb...",Data Source Selection in Big Data Context,"Big Data Source Selection, Data quality, Sourc...",Big Data presents promising technological and ...,2019,inproceedings,10.1145/3366030.3366121,ACM,366030.3366121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#Escrevendo o novo arquivo 

In [63]:
#csv
if 'csv' in arquivo_config['output']:
  arquivo_final.to_csv('resultado_csv_bibtex.csv')


In [64]:
#json
if 'json' in arquivo_config['output']:
  arquivo_final.to_json('resultado_csv_bibtex.json', orient="records")


In [65]:
#yaml
if 'yaml' in arquivo_config['output']:
  yaml_arquivos_entrada = yaml.dump(arquivo_final.to_dict("records"))
  escrita = open("resultado_csv_bibtex.yaml", "w")
  escrita.write(yaml_arquivos_entrada)
  escrita.close()

